# Segmenting and Clustering Neighborhoods in Toronto
------
## Part 1 - Creating dataset

This assignment is about data clustering on example of Toronto neigborhoods. Before we can start with machine learning algorithms, we have to create and prepare our dataset. This is what we will do in thist part. So after importing necessary libraries we will scarpe [wikipedia page](https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M) containing our data. To do so I have used pandas function `read_html`, which works like magic.

In [3]:
import pandas as pd 
import requests

In [4]:
link = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

r=requests.get(link)

tables = pd.read_html(r.text)
raw_df = tables[0]
raw_df.head()

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


We have to handle null values according to assignment. Even thought there assignment says that there could be some postal codes with borought assigned but neighbourhood not assigned, I have noticed that this doesn't occur in this case. Therefore part of code responsible for handling this situation won't have any impact on the dataset.

In [85]:
df = raw_df.copy().replace({'Not assigned': None})
df.dropna(subset=['Borough'],inplace=True)
df['Neighbourhood'][df['Neighbourhood'].isnull()] = df['Borough'][df['Neighbourhood'].isnull()]
df.reset_index(drop=True,inplace=True)
df.head(10)

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


In [89]:
df.shape

(103, 3)